In [1]:
from collections import defaultdict
import copy
import sys
from typing import Any, Callable, Dict, KeysView, List, Sequence, Tuple

DEBUG = False
def log(msg: str) -> None:
    if DEBUG:
        print(msg)

def sql(q: str) -> Dict[str, List[Any]]:
    # Don't parse SQL with regex, that's bad
    # Instead, split on whitespace!
    qws = q.lower().split()
    fromi = qws.index("from")
    joini = qws.index("join") if "join" in qws else len(qws)
    wherei = qws.index("where") if "where" in qws else len(qws)
    cols = qws[1:fromi]
    
    # FROM clause handling, pluck the relations from memory and enumerate them
    join = qws[joini:wherei]
    if len(join) > 0:
        from_tablename = qws[fromi:joini]
        join_tablename = qws[joini:wherei]
        join_clause = qws[joini+3:wherei]
        table = parse_join(from_tablename, join_tablename, join_clause)
    else:
        table = parse_relation(qws[fromi:joini])
    
    # WHERE clause handling
    filtered_table = table
    where_cols, where_func = parse_where(qws[wherei:], table.keys())
    if where_cols:
        tuple_table = zip(*[table[c].values() for c in where_cols])
        filtered_table = copy.deepcopy(table)
        for i, tup in enumerate(tuple_table):
            if not where_func(*tup):
                for k, v in filtered_table.items():
                    del v[i]
    
    # SELECT clause handling and removing the row index
    select_cols = parse_cols(cols)
    results = {}     
    for col in select_cols:
        results[col] = list(filtered_table[col].values())
    return results


def parse_join(from_tablename, join_tablename, join_clause):
    from_table = parse_relation(from_tablename)
    join_table = parse_relation(join_tablename)
    left, op, right = join_clause
    left_db, left_col = left.split(".")
    right_db, right_col = right.split(".")
   
    hashed_left = defaultdict(list)
    for i, v in from_table[left_col].items():
        hashed_left[v].append([v[i] for v in from_table.values()])
    if op == "=":
        ts = [tuple(ls) + tuple(jv[i] for jv in join_table.values()) for i, v in join_table[right_col].items() for ls in hashed_left[v]]
        unzipped = list(zip(*ts))
        cols = enumerate(list(from_table.keys()) + list(join_table.keys()))
        results = {col: {ii: vv for ii, vv in enumerate(unzipped[i])} for i, col in cols}
        return results
                
    return db_tables[from_tablename]
        

def parse_relation(relation: List[str]) -> Dict[str, Dict[int, Any]]:
    tablename = relation[1]
    try:
        table = getattr(sys.modules[__name__], tablename)
        dicty_table = {col: {i: v for i, v in enumerate(vals)} for col, vals in table.items()}
        log(f"parse_relation> found relation: {tablename}")
        return dicty_table
    except AttributeError:
        print(f"ERROR: Could not find relation '{tablename}'")
        raise

        
def parse_where(where: List[str], table_cols: KeysView[str]) -> Tuple[List[str], Callable[[Sequence], Any]]:
    if len(where) == 0:
        return ([], lambda x: x)
    where_func = " ".join(where[1:])
    used_columns = [tc for tc in table_cols if tc in where_func]
    lambda_string = f"lambda {', '.join(used_columns)}: {where_func}"
    log(f"parse_where> columns in WHERE clause: {used_columns}")
    log(f"parse_where> lambda string: {lambda_string}")
    return (used_columns, eval(lambda_string))
    
    
def parse_cols(cols: List[str]) -> List[str]:
    end = len(cols) - 1
    select_cols = []
    for i, col in enumerate(cols):
        if i == end or col.endswith(","):
            select_cols.append(col.rstrip(","))
        else:
            print("WARNING: aliases not yet supported: ", col)
    log(f"parse_cols> selection columns: {select_cols}")
    return select_cols

In [2]:
users = {
    "name": ["andrew", "dave", "alicia"],
    "food": ["pizza", "eggs", "poutine"]}
restaurants = {
    "place": ["Rugova's", "Willie's"],
    "food": ["pizza", "poutine"]}

In [3]:
sql("select name, food from users")

{'name': ['andrew', 'dave', 'alicia'], 'food': ['pizza', 'eggs', 'poutine']}

In [4]:
sql("""select name from users where name.startswith("a")""")

{'name': ['andrew', 'alicia']}

In [5]:
sql("""select name, food from users where food.endswith("t")""")

{'name': [], 'food': []}

In [6]:
sql("""select name, food from users where 'a' in food""")

{'name': ['andrew'], 'food': ['pizza']}

In [7]:
sql("""select name, food from users where 'egg' in food and name.startswith("d")""")

{'name': ['dave'], 'food': ['eggs']}

In [8]:
sql("""
select name, place, food
from users join restaurants on users.food = restaurants.food
""")

{'name': ['andrew', 'alicia'],
 'place': ["Rugova's", "Willie's"],
 'food': ['pizza', 'poutine']}

In [9]:
sql("""
select name, place, food
from users join restaurants on restaurants.food = users.food
where name.endswith("a")
""")

{'name': ['alicia'], 'place': ["Willie's"], 'food': ['poutine']}

In [10]:
invites = {
    "name": ["andrew", "dave", "alicia", "jon"],
    "status": ["accepted", "decline", "accepted", "accepted"]}
users = {
    "name": ["andrew", "dave", "alicia", "alicia", "jon"],
    "food": ["pizza", "eggs", "poutine", "pizza", "pizza"]}
places = {
    "place": ["Rugova's", "Willie's", "Weird Egg Bar"],
    "food": ["pizza", "poutine", "eggs"]}


# Who's coming to the party and what do they eat?
people_going = sql("""
    select name, food
    from users join invites on users.name = invites.name
    where status == "accepted"
    """)

# Where can we order from for the party?
# Note: Here we are joining against the results of the previous query!
orders = sql("""
    select name, place
    from places join people_going on places.food = people_going.food
    """)

print(orders)

{'name': ['andrew', 'alicia', 'alicia', 'jon'], 'place': ["Rugova's", "Willie's", "Rugova's", "Rugova's"]}
